In [18]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
%load_ext sql
%sql sqlite:///store.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


![scheme](scheme.png)

In [20]:
%%sql
SELECT * FROM sqlite_master;

 * sqlite:///store.db
Done.


type,name,tbl_name,rootpage,sql
table,albums,albums,2,"CREATE TABLE ""albums"" ( [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Title] NVARCHAR(160) NOT NULL, [ArtistId] INTEGER NOT NULL, FOREIGN KEY ([ArtistId]) REFERENCES ""artists"" ([ArtistId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,artists,artists,4,"CREATE TABLE ""artists"" ( [ArtistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,genres,genres,5,"CREATE TABLE ""genres"" ( [GenreId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,invoice_items,invoice_items,6,"CREATE TABLE ""invoice_items"" ( [InvoiceLineId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [InvoiceId] INTEGER NOT NULL, [TrackId] INTEGER NOT NULL, [Quantity] INTEGER NOT NULL, FOREIGN KEY ([InvoiceId]) REFERENCES ""invoices"" ([InvoiceId]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([TrackId]) REFERENCES ""tracks"" ([TrackId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,media_types,media_types,9,"CREATE TABLE ""media_types"" ( [MediaTypeId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,playlists,playlists,10,"CREATE TABLE ""playlists"" ( [PlaylistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,playlist_track,playlist_track,11,"CREATE TABLE ""playlist_track"" ( [PlaylistId] INTEGER NOT NULL, [TrackId] INTEGER NOT NULL, CONSTRAINT [PK_PlaylistTrack] PRIMARY KEY ([PlaylistId], [TrackId]), FOREIGN KEY ([PlaylistId]) REFERENCES ""playlists"" ([PlaylistId]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([TrackId]) REFERENCES ""tracks"" ([TrackId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
index,sqlite_autoindex_playlist_track_1,playlist_track,12,None
table,tracks,tracks,14,"CREATE TABLE ""tracks"" ( [TrackId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(200) NOT NULL, [AlbumId] INTEGER, [MediaTypeId] INTEGER NOT NULL, [GenreId] INTEGER, [Milliseconds] INTEGER NOT NULL, [Bytes] INTEGER, [UnitPrice] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([AlbumId]) REFERENCES ""albums"" ([AlbumId]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([GenreId]) REFERENCES ""genres"" ([GenreId]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([MediaTypeId]) REFERENCES ""media_types"" ([MediaTypeId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"


## Задание 1
Вывести 5 самых больших по размеру (в байтах) треков жанра Blues и формата MPEG. Только название и размер.


In [21]:
%%sql

SELECT 
  name, 
  bytes 
FROM 
  tracks t 
WHERE 
  t.genreid = (
    SELECT 
      genreid 
    FROM 
      genres g 
    WHERE 
      g.name = "Blues"
  ) 
  AND t.mediatypeid = (
    SELECT 
      mediatypeid 
    FROM 
      media_types m 
    WHERE 
      m.name = "MPEG audio file"
  ) 
ORDER BY 
  t.bytes DESC 
LIMIT 
  5;

 * sqlite:///store.db
Done.


Name,Bytes
Old Love,15780747
Running On Faith,12536275
Lonely Stranger,10894406
Layla,9490542
Tears In Heaven,9032835


## Задание 2
Вывести названия всех треков и их исполнителей для всех треков жанра Rock, приобретенных сотрудниками Oracle.


In [22]:
%%sql

SELECT 
  name, 
  (
    SELECT 
      name 
    FROM 
      artists a 
    WHERE 
      a.artistid = (
        SELECT 
          artistid 
        FROM 
          albums al 
        WHERE 
          al.albumid = t.albumid
      )
  ) artist
FROM 
  tracks t 
WHERE 
  t.genreid = (
    SELECT 
      genreid 
    FROM 
      genres g 
    WHERE 
      g.name = "Rock"
  ) 
  AND t.trackid IN (
    SELECT 
      trackid 
    FROM 
      invoice_items ii 
    WHERE 
      ii.invoiceid IN (
        SELECT 
          invoiceid 
        FROM 
          invoices i 
        WHERE 
          i.customerid IN (
            SELECT 
              customerid 
            FROM 
              customers c 
            WHERE 
              c.company = "Oracle"
          )
      )
  );

 * sqlite:///store.db
Done.


Name,artist
A Kind Of Magic,Queen
I Want To Break Free,Queen
Who Wants To Live Forever,Queen
The Invisible Man,Queen
Bad Moon Rising,Creedence Clearwater Revival
Down On The Corner,Creedence Clearwater Revival
Up Around The Bend,Creedence Clearwater Revival
That's The Way,Led Zeppelin
Ten Years Gone,Led Zeppelin
Achilles Last Stand,Led Zeppelin


## Задание 3
Для каждой компании вывести общее количество купленных джазовых композиций (если 0, можно не выводить).


In [23]:
%%sql

SELECT 
  c.company, 
  COUNT(t.trackid) jazz_tracks 
FROM 
  tracks t 
  LEFT JOIN genres g ON t.genreid = g.genreid 
  LEFT JOIN invoice_items ii ON t.trackid = ii.trackid 
  LEFT JOIN invoices i ON ii.invoiceid = i.invoiceid 
  LEFT JOIN customers c ON i.customerid = c.customerid 
WHERE 
  g.name = "Jazz" 
  AND c.company IS NOT NULL 
GROUP BY 
  c.company 
HAVING 
  jazz_tracks > 0;

 * sqlite:///store.db
Done.


Company,jazz_tracks
Apple Inc.,2
BMW,1
Google Inc.,12
JetBrains s.r.o.,3
Microsoft Corporation,15
Oracle,3
SAP,1
Telus,2


## Задание 4
Вывести топ 3 плейлиста по продолжительности (продолжительность плейлиста = сумма длин треков в нём). Только название и продолжительность (в часах).


In [24]:
%%sql

SELECT 
  name, 
  (
    SELECT 
      SUM(milliseconds) 
    FROM 
      tracks t 
    WHERE 
      t.trackid IN (
        SELECT 
          trackid 
        FROM 
          playlist_track pt 
        WHERE 
          pt.playlistid = p.playlistid
      )
  ) / 1000 / 3600 duration 
FROM 
  playlists p 
WHERE 
  duration > 0
ORDER BY 
  duration DESC 
LIMIT 
  3;


 * sqlite:///store.db
Done.


Name,duration
From That TV Show,138
Hidden Gems,138
90’s Music,10


## Задание 5
Для каждого набора (жанр, тип медиа) вывести количество треков в них, причем вывести только те наборы, для которых **все** треки стоят больше 1$ и для которых в наборе есть хотя бы один трек.

In [53]:
%%sql

WITH joined_tracks AS (
  SELECT 
    trackid, 
    unitprice price, 
    g.name genre, 
    mt.name filetype 
  FROM 
    tracks t 
    LEFT JOIN genres g ON t.genreid = g.genreid 
    LEFT JOIN media_types mt ON t.mediatypeid = mt.mediatypeid
) 
SELECT 
  genre, 
  filetype, 
  COUNT(trackid) tracks 
FROM 
  joined_tracks 
GROUP BY 
  genre, filetype
HAVING
  MIN(price) > 1 AND tracks > 0;

 * sqlite:///store.db
Done.


genre,filetype,tracks
Blues,AAC audio file,9
Comedy,MPEG audio file,17
Sci Fi & Fantasy,MPEG audio file,26
Science Fiction,MPEG audio file,13
TV Shows,MPEG audio file,93
